In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
def wrangle(filepath):
    df=pd.read_csv(filepath).set_index("id")
    df.drop(columns=["veil-type","spore-print-color","stem-root","veil-color","stem-surface","gill-spacing"],inplace=True)
    top_cap_shape=df["cap-shape"].value_counts().sort_values(ascending=False).head(27)
    df["cap-shape"]=df["cap-shape"].apply(lambda c : c if c in top_cap_shape else "other")
    top_cap_surface=df["cap-surface"].value_counts().sort_values(ascending=False).head(29)
    df["cap-surface"]=df["cap-surface"].apply(lambda x: x if x in top_cap_surface else "others")
    top_cap_color=df["cap-color"].value_counts().sort_values(ascending=False).head(29)
    df["cap-color"]=df["cap-color"].apply(lambda a : a if a in top_cap_color else "others")
    top_gill_attachment=df["gill-attachment"].value_counts().sort_values(ascending=False).head(25)
    df["gill-attachment"]=df["gill-attachment"].apply(lambda q : q if q in top_gill_attachment else "others")
    top_gill_color=df["gill-color"].value_counts().sort_values(ascending=False).head(32)
    df["gill-color"]=df["gill-color"].apply(lambda s: s if s in top_gill_color else "others")
    top_stem_color=df["stem-color"].value_counts().sort_values(ascending=False).head(26)
    df["stem-color"]=df["stem-color"].apply(lambda f: f if f in top_stem_color else "others")
    top_habitat=df["habitat"].value_counts().sort_values(ascending=False).head(27)
    df["habitat"]=df["habitat"].apply(lambda a : a if a in top_habitat else "others")
    top_ring_type=df["ring-type"].value_counts().sort_values(ascending=False).head(26)
    df["ring-type"]=df["ring-type"].apply(lambda j: j if j in top_ring_type else "others")
    top_bruises=df["does-bruise-or-bleed"].value_counts().sort_values(ascending=False).head(20)
    df["does-bruise-or-bleed"]=df["does-bruise-or-bleed"].apply(lambda m : m if m in top_bruises else "others")
    top_ring=df["has-ring"].value_counts().sort_values(ascending=False).head(18)
    df["has-ring"]=df["has-ring"].apply(lambda l : l if l in top_ring else "others")
    return df

In [4]:
df=wrangle("/kaggle/input/playground-series-s4e8/train.csv")
df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
0,e,8.80,f,s,u,f,a,w,4.51,15.39,w,f,f,d,a
1,p,4.51,x,h,o,f,a,n,4.79,6.48,o,t,z,d,w
2,e,6.94,f,s,b,f,x,w,6.85,9.93,n,f,f,l,w
3,e,3.88,f,y,g,f,s,g,4.16,6.53,w,f,f,d,u
4,e,5.85,x,l,w,f,d,w,3.37,8.36,w,f,f,g,a


In [5]:
df["class"]=df["class"].map({'e':1,'p':0})

In [6]:
df.shape

(3116370, 15)

# SPLIT

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X=df[['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color', 'stem-height',
       'stem-width', 'stem-color', 'has-ring', 'ring-type', 'habitat',
       'season']]
y=df["class"]

In [9]:
X_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
Baseline=y_train.value_counts(normalize=True).max()
Baseline

0.5470009979559551

# ITERATE

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import ConfusionMatrixDisplay,classification_report
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder

In [12]:
model=make_pipeline(
    OneHotEncoder(handle_unknown="ignore",use_cat_names=True),
    SimpleImputer(),
    DecisionTreeClassifier(max_depth=90,random_state=42)
)
model.fit(X_train,y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['cap-shape', 'cap-surface', 'cap-color',
                                     'does-bruise-or-bleed', 'gill-attachment',
                                     'gill-color', 'stem-color', 'has-ring',
                                     'ring-type', 'habitat', 'season'],
                               handle_unknown='ignore', use_cat_names=True)),
                ('simpleimputer', SimpleImputer()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=90, random_state=42))])

In [13]:
model.score(x_val,y_val)

0.9810211881130931

In [50]:
X_test=wrangle("/kaggle/input/playground-series-s4e8/test.csv")

In [52]:
X_test.shape

(2077964, 14)

In [53]:
preds=model.predict(X_test)

In [54]:
pred=pd.DataFrame({"id":X_test.index,"class":preds})

In [55]:
pred.shape

(2077964, 2)

In [56]:
pred["class"]=pred["class"].map({1:'e',0:'p'})

In [57]:
pred.shape

(2077964, 2)

In [58]:
pred.to_csv("mushroom_submission.csv",index=False)

In [59]:
ff=pd.read_csv("mushroom_submission.csv")

In [60]:
ff.head()

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e


In [61]:
ff.shape

(2077964, 2)